# Parameter sweeps

Parameter sweeps for testing clustering statistics.

## Parameters for sweeps

Here we define the overall parameters that will be used for running the analysis. The parameters include a `LABEL` to name the experiments and output data, the path to actually save the output data, sampling rates, cutoffs, as well as any other simulation parameter.

In [11]:
LABEL          = 'simple_parameter_sweeps'
RESULTS_PATH   = '../results'

SAMPLING_RATES = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
CUTOFFS        = [k*365 for k in [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

RAND_SEEDS = list( range(0, 100, 5) )

## Initialization

Imports and other environment configuration.

In [12]:
import os
#import time
#import warnings
#import multiprocessing

import numpy as np
import pandas as pd
#import seaborn
#from matplotlib import pyplot as plt
#%matplotlib inline

#from find_clusters import run_analysis

In [13]:
# Create output directory and file name prefix
results_dir = os.path.join( os.path.abspath(RESULTS_PATH), LABEL )
if os.path.exists( results_dir ):
    print( '... the directory ', results_dir )
    print( '    already exists. Simulation results may overwrite files in' )
    print( '    this directory.' )
    if input( '    Do you want to continue? [y/N] ' ) != 'y':
        raise Exception('stopping execution')
else:
    os.makedirs( results_dir )

output_prefix = results_dir + '/' + LABEL

... the directory  /workspaces/ClusterSim/results/simple_parameter_sweeps
    already exists. Simulation results may overwrite files in
    this directory.


## Load PANGEA-HIV clustering data

Let's get the data by reading this figure. We can use precise data (the one used for creating this figure) at a later time.
![PANGEA-HIV Clustering Results](pangea_clustering.png)

In [14]:
pangea_data = pd.DataFrame( [ [ 'HIVDRD, UK'       , 'red'  , 3.4, 1.5, 11  , 2  , 0.7 , 1.5  ],
                              [ 'PopART, Zambia'   , 'blue' , 2.3, 0.3,  3  , 0.4, 0.18, 3.19 ],
                              [ 'Rakai, Uganda'    , 'brown', 2.4, 0.3,  3  , 0.4, 0.18, 3.27 ],
                              [ 'Ya Tsie, Botswana', 'green', 2.1, 0.2,  2.5, 0.2, 0.05, 5.15 ]
                             ],
                             columns = [ 'location',
                                         'color',
                                         'cluster size (mean)',
                                         'cluster size (CoV)',
                                         'weighted cluster size (mean)',
                                         'weighted cluster size (CoV)',
                                         'Gini coefficient', 
                                         'R0'
                                        ]
                           )

## Run branching process model

In [15]:
partner_number_sweep   = [ 0.25, 0.5, 0.75 ]
lambda_parameter_sweep = [ 0.001, 0.002, 0.003 ]

In [16]:
raise

# Let's define the number of parallel processes and a wrapper function with that
# defines the code that a single process will execute
n_processes = max( int(multiprocessing.cpu_count()/2 - 2), 1 )

def analysis_worker( rand_seed ):
    params = SIM_PARAMS
    params['seed'] = rand_seed
    run_analysis( SAMPLING_RATES, CUTOFFS, params, output_prefix )
    return

# Sweep partner number
#for mean_partner_number in partner_number_sweep:

RuntimeError: No active exception to reraise